0. Importing Libraries

In [11]:
import pandas as pd
import numpy as np


1. Loading the data

In [199]:
# Loading the data of character metadata

df_characters = pd.read_csv('Data/character.metadata.tsv', sep='\t')
df_characters.columns = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date', 'Character name', 'Actor date of birth', 'Actor gender',
                        'Actor height (in meters)', 'Actor ethnicity (Freebase ID)', 'Actor name', 'Actor age at movie release',
                        'Freebase character/actor map ID', 'Freebase character ID', 'Freebase actor ID']

In [200]:
# Loading the data of movie metadata

df_movie = pd.read_csv('Data/movie.metadata.tsv', sep='\t')
df_movie.columns = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime',
                    'Movie languages (Freebase ID)', 'Movie countries (Freebase ID)', 'Movie genres (Freebase ID)']

In [220]:
# Loading name clusters
# 970 unique character names used in at least two different movies, along with 2,666 instances of those types

df_name_clusters = pd.read_csv('Data/name.clusters.txt', sep='\t')
df_name_clusters.columns = ['Character name', 'Freebase character/actor map ID']
print(len(df_characters))

450668


In [221]:
# Loading plot summaries
df_plot_summaries = pd.read_csv('Data/plot_summaries.txt', sep='\t')
df_plot_summaries.columns = ['Wikipedia movie ID', 'Plot Summary']
print(len(df_movie))

81740


2. Data Exploration

In [219]:
# Checking when either character name or Freebase movie ID are missing:
print('Missing "character names" in character metadata: %d' %df_characters['Character name'].isnull().sum())
print('Missing "Freebase character" IDs in character metadata: %d' %df_characters['Freebase character ID'].isnull().sum())
print('We can see that the number of missing character names ≈ missing Freebase character IDs')
print('There are 10 characters where we have a "Freebase character ID" but not "character names"')

# Checking when both "Character name" and "Freebase character ID are missing":
df_characters.loc[df_characters['Character name'].isnull() & df_characters['Freebase movie ID'].isnull()]
out = df_characters.iloc[:, [3, 11]].isnull().all(1)
out
true_count = sum(out)
print('Missing Character name AND Freebase character ID: %d' %true_count)

# We could try and recover the character names by their Freebase movie IDs.


Missing "character names" in character metadata: 257875
Missing "Freebase character" IDs in character metadata: 257865
We can see that the number of missing character names ≈ missing Freebase character IDs
There are 10 characters where we have a "Freebase character ID" but not "character names"
Missing Character name AND Freebase character ID: 257865


In [215]:
# Merging character with movie metadata with an inner join

df_char_movie = pd.merge(left=df_characters, right=df_movie, how='inner', on= ['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date'])
print('The total number of characters that we obtain from the inner merge is: %d' %len(df_char_movie))
print('By merging character and movie metadata we are missing')

The total number of characters that we obtain from the inner merge is: 450652


We do an inner join because there are movies in the movie metadata for which we do not have any character information in character metadata
and vice versa.

In [253]:
movies_with_characters = df_char_movie.groupby(['Wikipedia movie ID']).agg('size')
print('The resulting number of movies from the merge of characters and movies is: %d'%len(movies_with_characters))

The resulting number of movies from the merge of characters and movies is: 64329


In [18]:
movies = df_characters['Wikipedia movie ID']
characters = df_characters['Character name']

In [268]:
# Computing the total number of characters and the number of character names we have in our dataframe
char_names_per_movie = df_char_movie.groupby('Wikipedia movie ID')['Character name'].count() # extracts number of characters we have (i.e. not coun ting NaNs)
total_characters_per_movie = df_char_movie.groupby(['Wikipedia movie ID'])['Character name'].agg(['size'])

In [273]:
df_character_missing_stats = pd.merge(total_characters_per_movie, char_names_per_movie, on='Wikipedia movie ID')

df_character_missing_stats['n°_missing_characters'] = df_character_missing_stats['size'] - df_character_missing_stats['Character name']
df_character_missing_stats['percentage_missing'] = round(100 * df_character_missing_stats['n°_missing_characters'] / df_character_missing_stats['size'], 2) # computing percentages of missing character names
df_character_missing_stats = df_character_missing_stats.rename(columns={'size': 'total_n°_characters (incl. NaN)'})
df_character_missing_stats = df_character_missing_stats.rename(columns={'Character name': 'character_names (excl. NaN)'})
df_character_missing_stats

# In this dataframe we obtain the percentage of the character names that are missing per movie

,total_n°_characters (incl. NaN),character_names (excl. NaN),n°_missing_characters,percentage_missing
Wikipedia movie ID,,,,
330,4,0,4,100.00
3217,14,14,0,0.00
3333,14,0,14,100.00
3746,15,15,0,0.00
3837,20,16,4,20.00
...,...,...,...,...
37373877,7,2,5,71.43
37476824,10,0,10,100.00
37478048,4,1,3,75.00


Gathering some useful statistics about missing characters:

In [115]:
print(df_character_missing_stats['% of missing'][df_character_missing_stats['% of missing'] == 0].count()) # Have all names
print(df_character_missing_stats['Character Name (excl. NaN)'][df_character_missing_stats['Character Name (excl. NaN)'] == 0].count())
print(df_character_missing_stats['Character Name (excl. NaN)'][df_character_missing_stats['Character Name (excl. NaN)'] == 0].count()/64329)
print(df_character_missing_stats['% of missing'][df_character_missing_stats['% of missing'] <= 0.5].count()/64329)
print(df_character_missing_stats['Character Name (excl. NaN)'][df_character_missing_stats['Character Name (excl. NaN)'] >= 1].count())


# The 3rd number is the number of movies for which we are missing 100% of the character names. In other words we are missing all of the characters for roughly 50% of the movies.

# Plot histograms of percentages .....



9120
31759
0.4936964666013773
0.28463057097110167
32570


In [226]:
###################################################################################################

Creating a dataframe with the output of the NLP 

In [237]:
df_1 = pd.read_csv("Data/Plot_NLP_Analysis_0_14100.csv", sep='\t', header = None)
df_2 = pd.read_csv("Data/Plot_NLP_Analysis_14101_23201.csv", sep='\t', header = None)
df_3 = pd.read_csv("Data/Plot_NLP_Analysis_23202_42302.csv", sep='\t', header = None)
df_1 = df_1.drop(columns=0)
df_2 = df_2.drop(columns=0)
df_3 = df_3.drop(columns=0)

In [239]:
clusters_df = pd.concat([df_1, df_2, df_3], ignore_index=True)
clusters_df.columns = ('Wikipedia movie ID', 'Character name', 'Agent Verbs', 'Patient Verbs', 'Attributes')
clusters_df

,Wikipedia movie ID,Character name,Agent Verbs,Patient Verbs,Attributes
0,23890098,Lyosha,0.0,0.0,driver taxi
1,31186339,Primrose,0.0,['chosen'],old
2,31186339,Katniss,"['volunteers', 'survives', 'drops', 'has', 'ru...","['gave', 'taken', 'warned', 'presented']",sister love
3,31186339,Peeta,"['tribute', 'reveals', 'meant', 'forms', 'begs...",0.0,tribute mobile victorious
4,31186339,Haymitch,"['able', 'warns']",0.0,victor advice able star rule
...,...,...,...,...,...
150631,8628195,Amina,['forgotten'],0.0,daughter
150632,8628195,Rehman,"['abuses', 'tries', 'states', 'forgotten', 'tr...",['gives'],lawyer honesty daughter
150633,8628195,Mini,"['short', 'decides', 'goes', 'tolerate', 'stop...",0.0,father budget wedding
150634,6040782,Arthur,0.0,0.0,clerk


Trying to merge clusters_df df_char_movie

In [ ]:
#  Nerea Example from Homework 1
# df_comparison['positive1'] = np.where(df_comparison['headline1'].str.contains('|'.join(df_pos), 
#                     case=False, regex=True), 1, 0)

# Nerea example code for the merge

In [ ]:
#Example from the internet
#  df1
#     # check which LOCAL_NAME values are related
#     .assign(matches = lambda x: x["NAME"].apply(lambda y: df2.loc[df2["LOCAL_NAME"].str.contains(y), "LOCAL_NAME"].tolist()))
#     # make sure each row only has one key in the case that there are multiple matches found (is this possible?)
#     .explode("matches")
#     # join with the second dataframe to get the NAME_MAP column
#     .merge(df2, how="right", left_on="matches", right_on="LOCAL_NAME")
# )